In [156]:
import pandas as pd

total_data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv", sep = ",")
total_data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2


In [157]:
total_data.shape

(3140, 108)

Tenemos un data set muy grande con 108 columnas y 3140 filas

In [158]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB


In [159]:
print(total_data.duplicated().sum())

0


observamos que no hay filas duplicadas

como hay muchas columnas vamos a elminar la informacion irrelevante

In [160]:
total_data.columns

Index(['fips', 'TOT_POP', '0-9', '0-9 y/o % of total pop', '19-Oct',
       '10-19 y/o % of total pop', '20-29', '20-29 y/o % of total pop',
       '30-39', '30-39 y/o % of total pop',
       ...
       'COPD_number', 'diabetes_prevalence', 'diabetes_Lower 95% CI',
       'diabetes_Upper 95% CI', 'diabetes_number', 'CKD_prevalence',
       'CKD_Lower 95% CI', 'CKD_Upper 95% CI', 'CKD_number',
       'Urban_rural_code'],
      dtype='object', length=108)

In [161]:
unique_counts = total_data.nunique()
constant_columns = unique_counts[unique_counts == 1].index
constant_columns

Index([], dtype='object')

In [162]:

target_variable = "Heart disease_number"
y_= total_data[target_variable]

# Mostrar información sobre la variable objetivo
print(f"Información sobre {target_variable}:")
print(y.describe())

# Verificar los primeros registros de la variable objetivo
print(f"Primeros registros de {target_variable}:")
print(y.head())


Información sobre Heart disease_number:
count      3140.000000
mean       5597.593312
std       15615.285499
min           7.000000
25%         798.500000
50%        1814.500000
75%        4376.000000
max      434790.000000
Name: Heart disease_number, dtype: float64
Primeros registros de Heart disease_number:
0     3345
1    13414
2     2159
3     1533
4     4101
Name: Heart disease_number, dtype: int64


vamos a ver las  correlaciones para ver que variables podemos eliminar

In [163]:
string_columns = total_data.select_dtypes(include="object").columns
string_columns


Index(['COUNTY_NAME', 'STATE_NAME'], dtype='object')

In [164]:
total_data["COUNTY_NAME"] = pd.factorize(total_data["COUNTY_NAME"])[0]
total_data["STATE_NAME"] = pd.factorize(total_data["STATE_NAME"])[0]


In [165]:
correlation_matrix = total_data.corr()
target_correlation = correlation_matrix["Heart disease_number"].abs().sort_values(ascending=False)
correlation_matrix

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
fips,1.000000,-0.056079,-0.049580,-0.010586,-0.053130,0.018994,-0.053591,-0.030310,-0.051715,-0.045491,...,-0.063463,-0.031057,-0.036327,-0.026483,-0.056055,-0.040631,-0.044576,-0.033649,-0.062322,-0.008345
TOT_POP,-0.056079,1.000000,0.994010,0.052507,0.994596,0.023384,0.994558,0.160254,0.995526,0.295635,...,0.981495,-0.174843,-0.160621,-0.187362,0.989583,-0.218731,-0.202895,-0.230914,0.993889,-0.428608
0-9,-0.049580,0.994010,1.000000,0.085920,0.997487,0.048341,0.989992,0.161984,0.990971,0.304263,...,0.972641,-0.167495,-0.152687,-0.180585,0.984664,-0.216430,-0.200512,-0.229570,0.985337,-0.423641
0-9 y/o % of total pop,-0.010586,0.052507,0.085920,1.000000,0.074714,0.687844,0.056047,0.130136,0.058535,0.342930,...,0.034582,-0.028295,-0.005923,-0.048949,0.047539,-0.133841,-0.117138,-0.153847,0.040384,-0.050916
19-Oct,-0.053130,0.994596,0.997487,0.074714,1.000000,0.056762,0.988552,0.167605,0.987298,0.294682,...,0.973427,-0.178421,-0.163623,-0.191449,0.982778,-0.226956,-0.210885,-0.239865,0.985221,-0.431290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CKD_prevalence,-0.040631,-0.218731,-0.216430,-0.133841,-0.226956,-0.276999,-0.219593,-0.366835,-0.215739,-0.478115,...,-0.191943,0.918514,0.908023,0.925347,-0.175064,1.000000,0.995203,0.995594,-0.183045,0.471775
CKD_Lower 95% CI,-0.044576,-0.202895,-0.200512,-0.117138,-0.210885,-0.259547,-0.203506,-0.341480,-0.199999,-0.450913,...,-0.176039,0.924630,0.917499,0.928512,-0.159501,0.995203,1.000000,0.989756,-0.167424,0.454327
CKD_Upper 95% CI,-0.033649,-0.230914,-0.229570,-0.153847,-0.239865,-0.296576,-0.232199,-0.393541,-0.228048,-0.506720,...,-0.204368,0.906711,0.892914,0.916610,-0.187528,0.995594,0.989756,1.000000,-0.195052,0.489551
CKD_number,-0.062322,0.993889,0.985337,0.040384,0.985221,0.007654,0.986895,0.149367,0.985851,0.271343,...,0.991025,-0.144575,-0.131191,-0.156493,0.995536,-0.183045,-0.167424,-0.195052,1.000000,-0.416486


In [166]:
import pandas as pd
correlation_matrix = total_data.corr()

# Seleccionar las variables con menos correlación con la variable objetivo
target_variable = "Heart disease_number"
target_correlation = correlation_matrix[target_variable].abs().sort_values(ascending=True)
least_correlated_variables = target_correlation[:-1]  # Excluir la variable objetivo

# Mostrar las variables con menos correlación
print("Variables con menos correlación:")
print(least_correlated_variables.head(40))


Variables con menos correlación:
10-19 y/o % of total pop                                                         0.002344
R_DOMESTIC_MIG_2018                                                              0.014839
Active General Surgeons per 100000 Population 2018 (AAMC)                        0.016161
Unemployment_rate_2018                                                           0.020695
% Hawaiian/PI-alone                                                              0.029770
0-9 y/o % of total pop                                                           0.033228
COUNTY_NAME                                                                      0.040471
Percent of adults with less than a high school diploma 2014-18                   0.044691
% NA/AI-alone                                                                    0.047358
Active Patient Care General Surgeons per 100000 Population 2018 (AAMC)           0.051710
CNTY_FIPS                                                          

ELIMINO VARIABLES CON MENOS CORRELACION CON LA VARIABLE OBJETO

In [179]:

target_variable = "Heart disease_number"
target_correlation = correlation_matrix[target_variable].abs().sort_values(ascending=True)
least_correlated_columns = target_correlation.head(40).index

# Eliminar las 40 columnas menos correlacionadas del DataFrame
total_data_filtered = total_data.drop(columns=least_correlated_columns)
print("DataFrame después de eliminar las 40 columnas menos correlacionadas:")
print(total_data_filtered.head())

total_data_filtered.columns



DataFrame después de eliminar las 40 columnas menos correlacionadas:
   TOT_POP    0-9  19-Oct  20-29  30-39  30-39 y/o % of total pop  40-49  \
0    55601   6787    7637   6878   7089                 12.749771   7582   
1   218022  24757   26913  23579  25213                 11.564429  27338   
2    24881   2732    2960   3268   3201                 12.865239   3074   
3    22400   2456    2596   3029   3113                 13.897321   3038   
4    57840   7095    7570   6742   6884                 11.901798   7474   

   50-59  60-69  60-69 y/o % of total pop  ...  Heart disease_Upper 95% CI  \
0   7738   5826                 10.478229  ...                         8.7   
1  29986  29932                 13.728890  ...                         8.7   
2   3278   3076                 12.362847  ...                        11.8   
3   3115   2545                 11.361607  ...                         9.3   
4   7844   6965                 12.041840  ...                        10.1   

   He

Index(['TOT_POP', '0-9', '19-Oct', '20-29', '30-39',
       '30-39 y/o % of total pop', '40-49', '50-59', '60-69',
       '60-69 y/o % of total pop', '70-79', '80+', 'White-alone pop',
       'Black-alone pop', 'Native American/American Indian-alone pop',
       'Asian-alone pop', '% Asian-alone',
       'Hawaiian/Pacific Islander-alone pop', 'Two or more races pop',
       'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'GQ_ESTIMATES_2018',
       'R_INTERNATIONAL_MIG_2018', 'Less than a high school diploma 2014-18',
       'High school diploma only 2014-18',
       'Some college or associate's degree 2014-18',
       'Bachelor's degree or higher 2014-18',
       'Percent of adults with a high school diploma only 2014-18',
       'Percent of adults with a bachelor's degree or higher 2014-18',
       'POVALL_2018', 'MEDHHINC_2018', 'CI90LBINC_2018', 'CI90UBINC_2018',
       'Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018',
       'Median_Household_Income_2018', 'Total nurse pract

una vez eliminados las columanas menos relevantes seguimos estudiando y vamos a ver las mas correlacionadas con mi target

In [168]:
target_correlation = correlation_matrix[target_variable].abs().sort_values(ascending=False)
most_correlated_columns = target_correlation.head(40).index
most_correlated_columns

Index(['Heart disease_number', 'COPD_number', 'CKD_number',
       'anycondition_number', '60-69', 'High school diploma only 2014-18',
       'Population Aged 60+', '70-79', 'diabetes_number', 'Obesity_number',
       'county_pop2018_18 and older', 'POP_ESTIMATE_2018', 'TOT_POP', '50-59',
       'Total Population', 'Some college or associate's degree 2014-18',
       'Civilian_labor_force_2018', 'Employed_2018', '80+', '40-49',
       'White-alone pop', '19-Oct', '0-9', '20-29', '30-39', 'Unemployed_2018',
       'Family Medicine/General Practice Primary Care (2019)', 'POVALL_2018',
       'Total nurse practitioners (2019)',
       'Total Specialist Physicians (2019)',
       'Bachelor's degree or higher 2014-18',
       'Total physician assistants (2019)',
       'Internal Medicine Primary Care (2019)',
       'Less than a high school diploma 2014-18', 'Total Hospitals (2019)',
       'ICU Beds_x', 'GQ_ESTIMATES_2018', 'Two or more races pop',
       'Black-alone pop', 'Asian-alone po

In [169]:
total_data_filtered.isnull().sum().sort_values(ascending=False)/ len(total_data_filtered)
total_data_filtered.columns

Index(['TOT_POP', '0-9', '19-Oct', '20-29', '30-39',
       '30-39 y/o % of total pop', '40-49', '50-59', '60-69',
       '60-69 y/o % of total pop', '70-79', '80+', 'White-alone pop',
       'Black-alone pop', 'Native American/American Indian-alone pop',
       'Asian-alone pop', '% Asian-alone',
       'Hawaiian/Pacific Islander-alone pop', 'Two or more races pop',
       'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'GQ_ESTIMATES_2018',
       'R_INTERNATIONAL_MIG_2018', 'Less than a high school diploma 2014-18',
       'High school diploma only 2014-18',
       'Some college or associate's degree 2014-18',
       'Bachelor's degree or higher 2014-18',
       'Percent of adults with a high school diploma only 2014-18',
       'Percent of adults with a bachelor's degree or higher 2014-18',
       'POVALL_2018', 'MEDHHINC_2018', 'CI90LBINC_2018', 'CI90UBINC_2018',
       'Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018',
       'Median_Household_Income_2018', 'Total nurse pract

In [170]:
total_data_filtered.shape

(3140, 68)

no hay valores nulos

VAMOS A DIVIDIR EN TRAIN Y TEST ANTES DEL ESCALADO

In [171]:
total_data_filtered.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,30-39 y/o % of total pop,40-49,50-59,60-69,60-69 y/o % of total pop,...,Heart disease_Upper 95% CI,Heart disease_number,COPD_prevalence,COPD_Lower 95% CI,COPD_Upper 95% CI,COPD_number,diabetes_number,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,55601,6787,7637,6878,7089,12.749771,7582,7738,5826,10.478229,...,8.7,3345,8.6,7.3,9.9,3644,5462,3.3,1326,3
1,218022,24757,26913,23579,25213,11.564429,27338,29986,29932,13.728890,...,8.7,13414,8.6,7.2,10.1,14692,20520,3.5,5479,4
2,24881,2732,2960,3268,3201,12.865239,3074,3278,3076,12.362847,...,11.8,2159,12.1,10.7,13.3,2373,3870,4.8,887,6
3,22400,2456,2596,3029,3113,13.897321,3038,3115,2545,11.361607,...,9.3,1533,10.0,8.8,11.3,1789,2511,3.6,595,2
4,57840,7095,7570,6742,6884,11.901798,7474,7844,6965,12.041840,...,10.1,4101,10.5,9.1,12.0,4661,6017,3.7,1507,2


In [188]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import train_test_split
from pickle import dump, load

# Definir tu objeto scaler (por ejemplo, StandardScaler)
scaler = StandardScaler()

num_variables = ['TOT_POP', '0-9', '19-Oct', '20-29', '30-39', '30-39 y/o % of total pop', '40-49', '50-59', '60-69','60-69 y/o % of total pop', '70-79', '80+', 'White-alone pop','Black-alone pop', 'Native American/American Indian-alone pop',
'Asian-alone pop', '% Asian-alone',
'Hawaiian/Pacific Islander-alone pop', 'Two or more races pop',
'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'GQ_ESTIMATES_2018',
'R_INTERNATIONAL_MIG_2018', 'Less than a high school diploma 2014-18',
'High school diploma only 2014-18',
"Some college or associate's degree 2014-18",
"Bachelor's degree or higher 2014-18",
"Percent of adults with a high school diploma only 2014-18",
"Percent of adults with a bachelor's degree or higher 2014-18",'POVALL_2018', 'MEDHHINC_2018', 'CI90LBINC_2018', 'CI90UBINC_2018',
'Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018',
'Median_Household_Income_2018', 'Total nurse practitioners (2019)',
'Total physician assistants (2019)', 'Total Hospitals (2019)',
'Internal Medicine Primary Care (2019)',
'Family Medicine/General Practice Primary Care (2019)',
'Total Specialist Physicians (2019)', 'ICU Beds_x', 'Total Population',
'Population Aged 60+', 'Percent of Population Aged 60+','county_pop2018_18 and older', 'anycondition_prevalence',
'anycondition_Lower 95% CI', 'anycondition_Upper 95% CI',
'anycondition_number', 'Obesity_prevalence', 'Obesity_Lower 95% CI',
'Obesity_Upper 95% CI', 'Obesity_number', 'Heart disease_prevalence',
'Heart disease_Lower 95% CI', 'Heart disease_Upper 95% CI', 'COPD_prevalence', 'COPD_Lower 95% CI',
'COPD_Upper 95% CI', 'COPD_number', 'diabetes_number','CKD_Upper 95% CI', 'CKD_number', 'Urban_rural_code']

# We divide the dataset into training and test samples

X = total_data_filtered.drop("Heart disease_number", axis = 1)[num_variables]
y = total_data_filtered["Heart disease_number"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Guardar características del conjunto de entrenamiento
X_train.to_csv("../data/processed/train_X.csv", index=False)

# Guardar etiquetas del conjunto de entrenamiento
with open("../data/processed/train_y.txt", "w") as f:
    f.write(y_train.to_string(index=False))

# Guardar características del conjunto de prueba
X_test.to_csv("../data/processed/test_X.csv", index=False)

# Guardar etiquetas del conjunto de prueba en un archivo diferente
with open("../data/processed/test_y.txt", "w") as f:
    f.write(y_test.to_string(index=False))

# Guardar el objeto scaler
dump(scaler, open("../models/modelsinescala.pk", "wb"))
X_train.head()




,TOT_POP,0-9,19-Oct,20-29,30-39,30-39 y/o % of total pop,40-49,50-59,60-69,60-69 y/o % of total pop,...,Heart disease_Lower 95% CI,Heart disease_Upper 95% CI,COPD_prevalence,COPD_Lower 95% CI,COPD_Upper 95% CI,COPD_number,diabetes_number,CKD_Upper 95% CI,CKD_number,Urban_rural_code
1292,26625,3221,3463,2922,2829,10.625352,2831,3831,3860,14.497653,...,9.1,11.1,11.3,9.6,13.0,2314,2823,4.1,771,6
2302,51266,5272,5751,5137,5341,10.418211,5880,7885,8271,16.133500,...,8.2,10.1,9.9,8.3,11.5,4097,5416,3.8,1454,5
761,37779,3915,5118,6202,4363,11.548744,4451,5107,4349,11.511686,...,6.6,8.1,9.2,7.9,10.4,2792,3698,3.1,871,2
2194,91984,11163,12646,11595,11357,12.346712,11444,12774,10791,11.731388,...,7.1,8.5,8.1,7.1,9.3,5716,7913,3.2,2118,3
1241,134487,16698,17666,17281,15993,11.891856,15845,17982,16909,12.572962,...,7.7,9.1,9.6,8.4,11.0,10002,12987,3.6,3490,4


VAMOS A NORMALIZARLOS

In [190]:
from sklearn.preprocessing import StandardScaler
from pickle import dump

scaler = StandardScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_train_norm = pd.DataFrame(X_train_norm, index = X_train.index, columns = num_variables)

X_test_norm = scaler.transform(X_test)
X_test_norm = pd.DataFrame(X_test_norm, index = X_test.index, columns = num_variables)

X_train_norm.to_csv("../data/processed/train_X_norm.csv", index=False)
X_test_norm.to_csv("../data/processed/test_X_norm.csv", index=False)
dump(scaler, open("../data/processed/norm_scaler.pk", "wb"))

X_train_norm.head()

,TOT_POP,0-9,19-Oct,20-29,30-39,30-39 y/o % of total pop,40-49,50-59,60-69,60-69 y/o % of total pop,...,Heart disease_Lower 95% CI,Heart disease_Upper 95% CI,COPD_prevalence,COPD_Lower 95% CI,COPD_Upper 95% CI,COPD_number,diabetes_number,CKD_Upper 95% CI,CKD_number,Urban_rural_code
1292,-0.229763,-0.225393,-0.231350,-0.229775,-0.223780,-0.665485,-0.228216,-0.227030,-0.231375,0.538689,...,0.818916,0.874465,0.951061,0.856648,0.989521,-0.222477,-0.215940,0.657452,-0.216950,0.910528
2302,-0.161280,-0.179851,-0.181109,-0.188375,-0.176225,-0.785934,-0.163303,-0.139120,-0.112664,1.194600,...,0.255939,0.350729,0.350976,0.230367,0.413368,-0.117073,-0.135257,0.161947,-0.135212,0.249092
761,-0.198764,-0.209983,-0.195009,-0.168470,-0.194740,-0.128551,-0.193726,-0.199360,-0.218215,-0.658567,...,-0.744908,-0.696745,0.050934,0.037666,-0.009144,-0.194220,-0.188714,-0.994232,-0.204982,-1.735217
2194,-0.048115,-0.049041,-0.029705,-0.067671,-0.062335,0.335452,-0.044847,-0.033104,-0.044844,-0.570475,...,-0.432144,-0.487250,-0.420561,-0.347738,-0.431656,-0.021363,-0.057561,-0.829064,-0.055748,-1.073780
1241,0.070012,0.073864,0.080526,0.038603,0.025430,0.070962,0.048850,0.079829,0.119807,-0.233037,...,-0.056826,-0.173008,0.222387,0.278543,0.221317,0.232009,0.100319,-0.168390,0.108446,-0.412344
